# Purpose of this notebook is to extract, transform, and load the from the OpenFEC API the campaign (committee) funds spending data and committe attribute data in two tables in PostgresSQL 

### Import Libraries and Define Variables for API Parameters

In [3]:
import requests
import pandas as pd 
from pprint import pprint
from config import api_key
import os

url = "https://api.open.fec.gov/v1/schedules/schedule_b/efile/?"

In [220]:
committee_trump = "C00580100"
committee_biden = "C00703975"


### Make API Calls for Candidates and Append Results to Lists

In [179]:
# API Call for Candidate Trump 261 times at rate of 100 maximum records per call to get all results
resulted_t = []
for i in range (0,261):
    n_page = str(i+1)
    query_url_trump = url + "api_key=" + api_key + "&committee_id=" + committee_trump + "&per_page=100" + "&page=" + n_page
    disbursements_t = requests.get(query_url_trump).json()
    disbursements_t_list = disbursements_t["results"]
    resulted_t.append(disbursements_t_list)

261

In [185]:
# API Call for Candidate Biden 330 times at rate of 100 maximum records per call to get all results
resulted_b = []
for i in range (0,330):
    n_page = str(i+1)
    query_url_biden = url + "api_key=" + api_key + "&committee_id=" + committee_biden + "&per_page=100" + "&page=" + n_page
    disbursements_b = requests.get(query_url_biden).json()
    disbursements_b_list = disbursements_b["results"]
    resulted_b.append(disbursements_b_list)

### Create Committee Dictionaries from Results lists and merge into a Dataframe

In [212]:
# Create committee data dictionary for Trump
committee_keys_to_keep = ["committee_id", "name", "committee_type"]
# committee_dict = disbursements_t_list[0]['committee']
committee_dict = resulted_t[0][0]['committee']
committee_dic_t = dict((k, committee_dict[k]) for k in committee_keys_to_keep if k in committee_dict)

In [213]:
# Create committee data dictionary for Biden
committee_keys_to_keep2 = ["committee_id", "name", "committee_type"]
# committee_dict = disbursements_t_list[0]['committee']
committee_dict = resulted_b[0][0]['committee']
committee_dic_b = dict((k, committee_dict[k]) for k in committee_keys_to_keep2 if k in committee_dict)

In [215]:
# Combine the two committee dictionary and create a dataframe
def mergeDict(dict1, dict2):
    dict_3 = {**dict1, **dict2}
    for key, value in dict_3.items():
        if key in dict1 and key in dict2:
            dict_3[key] = [value , dict1[key]]
    return dict_3
dict_com = mergeDict(committee_dic_t, committee_dic_b)

committee = pd.DataFrame.from_dict(dict_com)
committee

,committee_id,name,committee_type
0,C00703975,BIDEN FOR PRESIDENT,P
1,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.",P


### Create Disbursements Dataframes from Results lists and merge into one Dataframe

In [222]:
# Filter out key disbursement attributes  for Candidate Trump into a Dataframe
dictionary_trump = {}
dictionary_trump["disbursement_date"] = []
dictionary_trump["recipient_name"] = []
dictionary_trump["recipient_state"] = []
dictionary_trump["committee_id"] = []
dictionary_trump["disbursement_description"] = []
dictionary_trump["disbursement_amount"] = []


for result in  resulted_t:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "disbursement_date":
                dictionary_trump["disbursement_date"].append(disbursevalue)
                      
for result in  resulted_t:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "recipient_name":
                dictionary_trump["recipient_name"].append(disbursevalue)

for result in  resulted_t:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "recipient_state":
                dictionary_trump["recipient_state"].append(disbursevalue)
            
for result in  resulted_t:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "committee_id":
                dictionary_trump["committee_id"].append(disbursevalue)
            
for result in  resulted_t:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "disbursement_description":
                dictionary_trump["disbursement_description"].append(disbursevalue)

for result in  resulted_t:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "disbursement_amount":
                dictionary_trump["disbursement_amount"].append(disbursevalue)
            
trump_info = pd.DataFrame.from_dict(dictionary_trump)

In [223]:
# Filter out key disbursement attributes  for Candidate Biden into a Dataframe
dictionary_biden = {}
dictionary_biden["disbursement_date"] = []
dictionary_biden["recipient_name"] = []
dictionary_biden["recipient_state"] = []
dictionary_biden["committee_id"] = []
dictionary_biden["disbursement_description"] = []
dictionary_biden["disbursement_amount"] = []


for result in  resulted_b:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "disbursement_date":
                dictionary_biden["disbursement_date"].append(disbursevalue)
            
for result in  resulted_b:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "recipient_name":
                dictionary_biden["recipient_name"].append(disbursevalue)

for result in  resulted_b:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "recipient_state":
                dictionary_biden["recipient_state"].append(disbursevalue)
            
for result in  resulted_b:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "committee_id":
                dictionary_biden["committee_id"].append(disbursevalue)
            
for result in  resulted_b:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "disbursement_description":
                dictionary_biden["disbursement_description"].append(disbursevalue)
            
for result in  resulted_b:                 
    for disbursements in result:                 
        for disburse, disbursevalue in disbursements.items():
            if disburse == "disbursement_amount":
                dictionary_biden["disbursement_amount"].append(disbursevalue)
            
biden_info = pd.DataFrame.from_dict(dictionary_biden)

In [224]:
# Combie two dataframes into one dataframe
candidatepds = [trump_info, biden_info]
spending = pd.concat(candidatepds)
spending

,disbursement_date,recipient_name,recipient_state,committee_id,disbursement_description,disbursement_amount
0,2020-10-14,"LEGENDARY CAMPAIGNS, LLC",DC,C00580100,ONLINE ADVERTISING,80.85
1,2020-10-14,STRIPE,CA,C00580100,MERCHANT FEES,0.77
2,2020-10-14,WINRED TECHNICAL SERVICES LLC,VA,C00580100,MERCHANT FEES,227550.23
3,2020-10-14,STRIPE,CA,C00580100,MERCHANT FEES,5133.02
4,2020-10-14,"EVENT RESOURCE GROUP, INC.",FL,C00580100,AUDIO VISUAL SERVICES,113207.74
...,...,...,...,...,...,...
32973,2020-03-18,Avis Rent A Car,NJ,C00703975,Vehicle Rental,197.43
32974,2020-02-14,American Express,NY,C00703975,Credit Card Fees,90.00
32975,2020-02-14,American Express,NY,C00703975,Credit Card Fees,67.50
32976,2020-02-14,American Express,NY,C00703975,Credit Card Fees,90.00


In [4]:
output_file = os.path.join("Output_API", "spending_df.csv")

spending.to_csv(output_file, index=False, header=True)

NameError: name 'spending' is not defined

### Connect to Local Postgres SQL database and Check for tables

In [ ]:
# Connect to local database
pg_user = 'postgres'
pg_password = 'postgres'
db_name = 'Candidate'


connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Check for tables
engine.table_names()

### Load Dataframes into Database and Confirm data has been added

In [ ]:
### Use Pandas to load Dataframes into database
spending.to_sql(name='Candidate_Spending', con=engine, if_exists='append', index=False)
committee.to_sql(name='Candidates', con=engine, if_exists='append', index=False)


In [ ]:
# Confirm both tables have been added
pd.read_sql_query('select * from Candidate_Spending', con=engine).head()
pd.read_sql_query('select * from Candidates', con=engine).head()